In [1]:
import pandas as pd


In [ ]:
votacao = pd.read_excel('../TabelaVotacaoFull.xlsx')
presenca = pd.read_excel('..TabelaPresencaFull.xlsx')

In [ ]:
votacao.head()

In [ ]:
votacao.rename(columns={'CPF Full': 'CPF'}, inplace=True)

# Remover os caracteres '-' e '/' da coluna 'CPF Full'
votacao['CPF'] = votacao['CPF'].str.replace(r'[-/,.]', '', regex=True)
votacao['Nome'] = votacao['Nome'].str.replace(r'\|', 'I', regex=True)

In [ ]:
presenca.head()

,Unnamed: 0,Nome,Documento,Classe,Valor,Representante,CPF Representante,OAB,Data
0,0,ANDRE PEREIRA DAS NEVES,97613550478,Trabalhista,"R$ 160.971,75",André Pereira das Neves,97613550478,NaN,05/11/2024 11:46:22
1,2480,I. A. DOS SANTOS LOCACOES - ME,NaN,ME e EPP,"R$ 314.905,76",MARCONDES MAGALHÃES ASSUNÇÃO,03531257331,OAB/PI 10.730 E OAB/MA 15.154-A,05/11/2024 11:56:16
2,3,EDIANE DA SILVA NASCIMENTO,357267158,Trabalhista,"R$ 15.246,84",THIAGO VITORINO DE ARAUJO,685414302,OAB CE 26.140,05/11/2024 11:20:21
3,1,CARINE DE LIMA SOBREIRA FERREIRA,5245996707,Trabalhista,"R$ 381.986,26",CARINE DE LIMA SOBREIRA FERREIRA,5245996707,NaN,05/11/2024 11:26:02
4,2,LORETTA KNETT,25792784434,Trabalhista,"R$ 2.451.692,40",ADRIANA MELLO OLIVEIRA DE CAMPOS MACHADO,158116704,OAB PE 16.331,05/11/2024 14:36:14
...,...,...,...,...,...,...,...,...,...
5434,5406,ANTONIO DE MORAIS ALBUQUERQUE,7035015426,Trabalhista,"R$ 26.147,90",CAROLINA SILVESTRE DE MATOS,05111709479,OAB/PE 26.142,05/11/2024 19:20:21
5435,5419,ANTONIO FRANCISCO ALVES DA ROCHA,2070771423,Trabalhista,"R$ 431.931,86",EROS SAFH DOMINGUES DA SILVA,43180418400,OAB/PE n.º 17.816,05/11/2024 11:49:20
5436,5430,ANTONIO GALDINO PESSOA NETO,4787049410,Trabalhista,"R$ 53.982,43",CAROLINA SILVESTRE DE MATOS,05111709479,OAB/PE 26.142,05/11/2024 19:20:21
5437,5434,ANTONIO GONCALVES DE LIMA,19030649453,Trabalhista,"R$ 11.600,32",CAROLINA SILVESTRE DE MATOS,05111709479,OAB/PE 26.142,05/11/2024 19:20:21


In [ ]:
presenca.rename(columns={'Documento': 'CPF'}, inplace=True)

# Remover os caracteres '-' e '/' da coluna 'CPF Full'
presenca['CPF'] = presenca['CPF'].str.replace(r'[-/,.]', '', regex=True)
presenca['Nome'] = presenca['Nome'].str.replace(r'\|', 'I', regex=True)

In [ ]:
from rapidfuzz import fuzz, process

In [ ]:
def fuzzy_join(df1, df2, key1, key2, threshold=90):
    matches = []
    for idx, value in df1[key1].items():
        # Encontrar a melhor correspondência para cada valor
        best_match = process.extractOne(value, df2[key2], scorer=fuzz.token_sort_ratio)
        if best_match and best_match[1] >= threshold:  # Similaridade acima do limite
            matched_idx = df2[df2[key2] == best_match[0]].index[0]
            matches.append((idx, matched_idx))
    return matches

# Realizar o join com base na similaridade de nomes
matches = fuzzy_join(presenca, votacao, key1='Nome', key2='Nome', threshold=90)

# Unir os DataFrames com base nas correspondências
presenca['match_index'] = None
for pres_idx, vot_idx in matches:
    presenca.loc[pres_idx, 'match_index'] = vot_idx

result = presenca.merge(votacao, left_on='match_index', right_index=True, how='left')
result.drop(columns=['match_index'], inplace=True)

# Exibir o resultado
print(result)

      Unnamed: 0                               Nome_x        CPF_x  \
0              0              ANDRE PEREIRA DAS NEVES  97613550478   
1           2480       I. A. DOS SANTOS LOCACOES - ME          NaN   
2              3           EDIANE DA SILVA NASCIMENTO    357267158   
3              1     CARINE DE LIMA SOBREIRA FERREIRA   5245996707   
4              2                        LORETTA KNETT  25792784434   
...          ...                                  ...          ...   
5434        5406        ANTONIO DE MORAIS ALBUQUERQUE   7035015426   
5435        5419     ANTONIO FRANCISCO ALVES DA ROCHA   2070771423   
5436        5430          ANTONIO GALDINO PESSOA NETO   4787049410   
5437        5434            ANTONIO GONCALVES DE LIMA  19030649453   
5438        5436  ANTONIO HELENO ALVES DA SILVA FILHO   3202651455   

         Classe_x            Valor                             Representante  \
0     Trabalhista    R$ 160.971,75                   André Pereira das Neves   

In [ ]:
result.to_excel("resultado_limpo.xlsx", index=False)